In [178]:
import pandas as pd
# from jinja2 import Environment, FileSystemLoader
from docxtpl import DocxTemplate
from datetime import datetime
import math

Set Up

In [179]:
data_file = "COAST_Data_9_25_23.xlsx"
study_id = "84HZC"

In [180]:
data_df = pd.read_excel(data_file, index_col=False)
participant_df = data_df[data_df["participant_id"] == study_id]

In [181]:
participant_dict = participant_df.to_dict("records")[0]
report_dict = {}

Demographic Info

In [182]:
first_name = participant_dict["qq_first_name"]
report_dict.update({"first_name": first_name})
last_name = participant_dict["qq_last_name"]
report_dict.update({"last_name": last_name})
start_date = str(participant_dict["qq_start_date"]).split(" ")[0]
report_dict.update({"start_date": start_date})

In [187]:
if participant_dict["qq_covid_number"] and participant_dict["qq_covid_number"] != "I have never had COVID-19":
    covid_number = int(participant_dict["qq_covid_number"])
else:
    covid_number = 0
report_dict.update({"covid_number": covid_number})

In [185]:
if participant_dict["qq_covid_number"] and not math.isnan(participant_dict["qq_covid_number"]) and participant_dict["qq_covid_number"] != "I have never had COVID-19":
    covid_number = int(participant_dict["qq_covid_number"])
else:
    covid_number = 0
report_dict.update({"covid_number": covid_number})

TypeError: must be real number, not str

In [189]:
def format_symptom(symptom):
    return symptom.replace("_", " ").title()

In [190]:
covid_symptoms = [
    "nasal_congestion",
    "sore_throat",
    "runny_nose",
    "ear_pain",
    "cough",
    "sputum_production",
    "difficulty_breathing_sob",
    "hoarse_voice",
    "chest_pain_tightness",
    "chills",
    "swollen_lymph_nodes",
    "skipping_meals_appetite_loss",
    "insomnia_sleep_problems",
    "sensitivity_heat_cold",
    "sweats",
    "white_red_purple_swollen_fingers_toes",
    "fever_feverish",
    "fatigue",
    "mood_swings_irritability",
    "weight_loss",
    "drowsiness",
    "tachycardia_arrhythmia_palpitations",
    "eye_soreness_discomfort",
    "reduced_blurred_vision",
    "photophobia_phonophobia",
    "brain_fog",
    "confusion",
    "memory_problems",
    "difficulty_concentrating",
    "delirium",
    "difficulty_finding_words",
    "paresthesia",
    "headache",
    "los",
    "lot",
    "dizziness_lightheadedness",
    "difficulty_balancing",
    "tremors",
    "stroke",
    "seizures",
    "hypoacusis",
    "numbness_hands_feet",
    "hypoethesia",
    "abdominal_pain_stomachache",
    "diarrhea",
    "nausea_vomiting",
    "muscle_weakness",
    "muscle_pains_aches",
    "bone_joint_pain",
    "neck_back_pain"
]

In [191]:
covid_rows = []
if covid_number:
    for i in range(covid_number):
        row_dict = {}
        incidence_num = i + 1
        if not participant_dict[f"qq_covid_{incidence_num}_symptom_onset_month"]:
            row_dict.update({"label": str(incidence_num), "date": "No Symptom Onset", "symptoms": "N/A - No Ongoing Symptoms"})
            covid_rows.append(row_dict)
        month = str(int(participant_dict[f"qq_covid_{incidence_num}_symptom_onset_month"]))
        day = str(int(participant_dict[f"qq_covid_{incidence_num}_symptom_onset_day"])) 
        year = str(int(participant_dict[f"qq_covid_{incidence_num}_symptom_onset_year"]) + 1899)
        date = f"{month}/{day}/{year}"
        symptom_list = []
        for symptom in covid_symptoms:
            if participant_dict[f"qq_covid_{incidence_num}_duration_{symptom}"] == "I am still experiencing this symptom":
                symptom_list.append(format_symptom(symptom))
        symptom_str = ", ".join(symptom_list)
        if not symptom_str:
            symptom_str = "N/A - No Ongoing Symptoms"
        row_dict.update({"label": str(incidence_num), "date": date, "symptoms": symptom_str})
        covid_rows.append(row_dict)
    report_dict.update({"covid_rows": covid_rows})
else:
    report_dict.update({"covid_rows": [{"label": "", "date": "", "symptoms": "No Reported History of COVID-19"}]})

In [192]:
tbi_symptoms = [
    "headache",
    "nausea",
    "vomiting",
    "balance_problems",
    "dizziness",
    "lightheadedness",
    "fatigue",
    "trouble_falling_asleep",
    "sleeping_more",
    "sleeping_less",
    "drowsiness",
    "light_sensitivity",
    "noise_sensitivity",
    "irritability",
    "feeling_frustrated_impatient",
    "taking_longer_to_think",
    "restlessness",
    "sadness",
    "nervousness_anxiousness",
    "feeling_more_emotional",
    "numbness_tingling",
    "feeling_slowed_down",
    "in_a_fog",
    "difficulty_concentrating",
    "difficulty_remembering",
    "blurred_vision",
    "double_vision",
    "pain"
]

In [193]:
if participant_dict["qq_tbi_num"] and not math.isnan(participant_dict["qq_tbi_num"]):
    tbi_number = int(participant_dict["qq_tbi_num"])
else:
    tbi_number = 0
report_dict.update({"tbi_number": tbi_number})

In [194]:
tbi_rows = []
if tbi_number:
    for i in range(tbi_number):
        row_dict = {}
        incidence_num = i + 1
        month = str(participant_dict[f"qq_tbi_{incidence_num}_month"])
        year = str(int(participant_dict[f"qq_tbi_{incidence_num}_year"]))
        date = f"{month} {year}"
        symptom_list = []
        for symptom in tbi_symptoms:
            if participant_dict[f"qq_tbi_{incidence_num}_duration_{symptom}"] == "I am still experiencing this symptom":
                symptom_list.append(format_symptom(symptom))
        symptom_str = ", ".join(symptom_list)
        if not symptom_str:
            symptom_str = "N/A - No Ongoing Symptoms"
        row_dict.update({"label": str(incidence_num), "date": date, "symptoms": symptom_str})
        tbi_rows.append(row_dict)
    report_dict.update({"tbi_rows": tbi_rows})
else:
    report_dict.update({"tbi_rows": [{"label": "", "date": "", "symptoms": "No Reported History of Brain Injury"}]})

In [ ]:
vaccine_dose_cols = [
    "qq_covid_vaccination_doses___1",
    "qq_covid_vaccination_doses___2",
    "qq_covid_vaccination_doses___3",
    "qq_covid_vaccination_doses___4",
]
label_dict = {
    "1": "First Dose",
    "2": "Second Dose",
    "3": "Third Dose",
    "4": "Booster"
}
vaccine_incidence_list = []
for col in vaccine_dose_cols:
    if participant_dict[col] == "Checked":
        vaccine_incidence_list.append(col[-1])

vaccine_number = len(vaccine_incidence_list)
report_dict.update({"vaccine_number": vaccine_number})

vaccine_rows = []
if vaccine_incidence_list:
    for num in vaccine_incidence_list:
        row_dict = {}
        if num == "4":
            num =  "booster"
        month = str(int(participant_dict[f"qq_covid_vaccination_dose_{num}_date_month"]))
        day = str(int(participant_dict[f"qq_covid_vaccination_dose_{num}_date_day"]))
        year = str(int(participant_dict[f"qq_covid_vaccination_dose_{num}_date_year"]) + 1899)
        date = f"{month}/{day}/{year}"
        if num == "4":
            vaccine_type_col = "qq_covid_vaccine_type_booster"
        else:
            vaccine_type_col = f"qq_covid_vaccine_type_dose_{num}"
        vaccine_type = participant_dict[vaccine_type_col]
        if not participant_dict[vaccine_type_col]:
            vaccine_type = participant_dict[f"qq_covid_vaccine_type_dose_{num}_other"]
        else:
            vaccine_type = participant_dict[vaccine_type_col]
        row_dict.update({"label": label_dict[num], "date": date, "type": vaccine_type})
        vaccine_rows.append(row_dict)
    report_dict.update({"vaccine_rows": vaccine_rows})
else:
    report_dict.update({"vaccine_rows": [{"label": "", "date": "No Vaccination History", "type": ""}]})


In [195]:
doc = DocxTemplate("coast_report_template.docx")

In [196]:
doc.render(report_dict)

In [197]:
today = datetime.today().strftime('%Y-%m-%d')
doc.save(f"reports/{last_name}_{first_name}_y1_qq_report_{today}.docx")

In [198]:
num = 0
if num:
    print("hi")